Based one

https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Gemma3_(4B).ipynb

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.float16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

# More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name = "unsloth/Llama-3.2-1B-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
    model_name = "meta-llama/Llama-3.2-3B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# num_added_toks = tokenizer.add_tokens(['<|space|>'], special_tokens=True) ##This line is updated
# model.resize_token_embeddings(len(tokenizer))



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Quadro RTX 5000. Num GPUs = 1. Max memory: 16.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Load LoRA adapters

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128

    #https://docs.unsloth.ai/basics/continued-pretraining#continued-pretraining-and-finetuning-the-lm_head-and-embed_tokens-matrices
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [3]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.2",
)

In [4]:
from datasets import load_dataset
dataset = load_dataset("philipfourie/books3_basic_sentenses_paraphrased-Morse", split="train").shuffle(seed=3408).select(range(40000)) # Load a small subset of the
dataset

Dataset({
    features: ['line', 'morse'],
    num_rows: 40000
})

In [5]:
dataset[0]

{'line': 'There was rain and then there was modesty to the shipping spirit.',
 'morse': '- .... . .-. . / .-- .- ... / .-. .- .. -. / .- -. -.. / - .... . -. / - .... . .-. . / .-- .- ... / -- --- -.. . ... - -.-- / - --- / - .... . / ... .... .. .--. .--. .. -. --. / ... .--. .. .-. .. - .-.-.-'}

In [6]:
def apply_conversations(example):
    return {
        "conversations": [
            {
                "content": example["line"], #+ ">>",
                "role": "user",
            },
            {
                #"content": example["morse"].replace(" ", "<|space|>") + tokenizer.eos_token,
                "content": example["morse"] + tokenizer.eos_token,
                "role": "assistant",
            },
        ]
    }

In [7]:
convos = dataset.map(
    apply_conversations,
    remove_columns = ["line", "morse"],
    #num_proc = 1,
    desc = "Applying conversations",
)
convos

Dataset({
    features: ['conversations'],
    num_rows: 40000
})

In [8]:
convos[0]

{'conversations': [{'content': 'There was rain and then there was modesty to the shipping spirit.',
   'role': 'user'},
  {'content': '- .... . .-. . / .-- .- ... / .-. .- .. -. / .- -. -.. / - .... . -. / - .... . .-. . / .-- .- ... / -- --- -.. . ... - -.-- / - --- / - .... . / ... .... .. .--. .--. .. -. --. / ... .--. .. .-. .. - .-.-.-<|end_of_text|>',
   'role': 'assistant'}]}

In [9]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

convos = convos.map(formatting_prompts_func, remove_columns=["conversations"], batched = True)
convos

Dataset({
    features: ['text'],
    num_rows: 40000
})

In [10]:
convos["text"][0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nThere was rain and then there was modesty to the shipping spirit.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n- .... . .-. . / .-- .- ... / .-. .- .. -. / .- -. -.. / - .... . -. / - .... . .-. . / .-- .- ... / -- --- -.. . ... - -.-- / - --- / - .... . / ... .... .. .--. .--. .. -. --. / ... .--. .. .-. .. - .-.-.-<|end_of_text|><|eot_id|>'

In [11]:
# split the dataset into train and test
train_dataset = convos.train_test_split(test_size=0.1, seed=42)["train"]
test_dataset = convos.train_test_split(test_size=0.1, seed=42)["test"]

convos = None
dataset = None

In [12]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

test_dataset = test_dataset.shuffle(seed=3408).select(range(1000))

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    max_seq_length = max_seq_length,
    #data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False, # Can make training 5x faster for short sequences.

    args = SFTConfig(
        dataset_text_field="text",
        output_dir="outputs",  # Explicitly set output directory
        logging_dir="outputs/tensorboard_logs",  # Subdirectory for TensorBoard logs
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!

        warmup_steps = 15,
        #num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 18301, #1301
        #max_steps = 100,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps=1,  # Log every step for testing
        eval_steps=325,  # Enable evaluation logging
        eval_strategy="steps",        
        
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3408,
        report_to = "tensorboard", # Use this for WandB etc
        dataset_num_proc=8,

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),        
    ),
)

In [13]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [14]:
trainer.train_dataset[100]["input_ids"]

[128000,
 128000,
 128006,
 9125,
 128007,
 271,
 38766,
 1303,
 33025,
 2696,
 25,
 6790,
 220,
 2366,
 18,
 198,
 15724,
 2696,
 25,
 220,
 1627,
 5887,
 220,
 2366,
 19,
 271,
 128009,
 128006,
 882,
 128007,
 271,
 6854,
 5127,
 1518,
 420,
 30,
 2684,
 596,
 459,
 469,
 5636,
 1828,
 311,
 4395,
 0,
 128009,
 128006,
 78191,
 128007,
 271,
 12,
 12898,
 13,
 662,
 12,
 43845,
 611,
 662,
 2564,
 12,
 662,
 662,
 14863,
 482,
 33501,
 12730,
 43845,
 662,
 611,
 2564,
 662,
 662,
 611,
 482,
 22666,
 5354,
 2564,
 5354,
 313,
 497,
 611,
 482,
 22666,
 662,
 662,
 14863,
 662,
 662,
 381,
 13,
 2564,
 611,
 662,
 12,
 43845,
 611,
 662,
 611,
 662,
 313,
 13,
 662,
 12,
 497,
 5354,
 12,
 2564,
 611,
 43845,
 662,
 482,
 497,
 12,
 482,
 611,
 482,
 12730,
 611,
 662,
 2564,
 12,
 662,
 662,
 14863,
 482,
 33501,
 482,
 22666,
 5354,
 43845,
 1198,
 13,
 482,
 12898,
 33501,
 128001,
 128009]

In [15]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                  -.-..- -. /....-..-. -.-- --- -.. /..... / -...........--.. / -......-...----.... /.- -. /. /.--..-....-... / -.. -..- - / - --- /....-..-. -.-- -...... -. --. -.-.--<|end_of_text|><|eot_id|>'

In [16]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Quadro RTX 5000. Max memory = 16.0 GB.
6.779 GB of memory reserved.


In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 36,000 | Num Epochs = 1 | Total steps = 18,301
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 12,156,928/3,224,906,752 (0.38% trained)


Step,Training Loss,Validation Loss
325,1.654600,1.570968
650,1.618800,1.426326
975,1.349700,1.326606
1300,1.507700,1.286474
1625,1.132600,1.223945
1950,1.083600,1.124720
2275,1.106500,0.843331
2600,0.335100,0.577531
2925,0.349800,0.431750
3250,0.326500,0.375976


Unsloth: Will smartly offload gradients to save VRAM!


In [41]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.2",
    
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [{
    "role": "user",
    "content":  "Nothing is working as expected.",
}]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")


outputs = model.generate(input_ids = inputs, 
                         max_new_tokens = 200,
                           use_cache = False,
                         temperature = 0.01)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nNothing is working as expected.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n-. --- -...... -. --. /..... /.-- ---.-. -.-.. -. --. /.-... /. -..-.--.. -.-. -. -...-.-.-<|end_of_text|>']

In [19]:
# from unsloth.chat_templates import get_chat_template
# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "llama-3.2",
# )
# messages = [{
#     "role": "user",
#     "content": [{
#         "type" : "text",
#         "text" : "The cat sat on the>>",
#     }]
# }]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize= False,
#     add_generation_prompt = True, # Must add for generation
# )


# outputs = model.generate(
#     **tokenizer([text], return_tensors = "pt").to("cuda"),
#     max_new_tokens = 120, # Increase for longer outputs!
#     # Recommended Gemma-3 settings!
#     temperature = 0.01, top_p = 0.95, top_k = 64,
# )

# output  = tokenizer.batch_decode(outputs)[0]
# print(output)

# from decode import decode_from_morse

# import re

# match = re.search(r'<start_of_turn>model\n(.*?)<end_of_turn>', output, re.DOTALL)
# if match:
#     result = match.group(1)
#     #print(result)
#     print(decode_from_morse(result))
# else:
#     print("No match found.")
    

In [20]:
#model.save_pretrained("gemma-3-1b-sos")  # Local saving
#tokenizer.save_pretrained("gemma-3-1b-sos")

In [21]:
if False: # Change to True to save finetune!
    #model.save_pretrained_merged("gemma-3-finetune2", tokenizer, save_method = "merged_4bit")
    #odel.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit",)
    #odel.save_pretrained_gguf("dir", tokenizer, quantization_method = "Q8_0")
    model.push_to_hub_gguf("philipfourie/gemma-3-finetune",  quantization_type ="q8_0")

In [22]:
if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "philipfourie/gemma-3-1b-morse-code", tokenizer,
    )

In [23]:
# model.save_pretrained_gguf(
#     "gemma-3-1b-morse-code",
#     quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
# )

In [24]:
# if True: # Change to True to upload GGUF
#     model.push_to_hub_gguf(
#         #"gemma-3-1b-morse-code",
#         quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
#         repo_id = "philipfourie/gemma-3-1b-morse-code"
#     )

In [25]:
#model.save_pretrained("lora_model")  # Local saving
#tokenizer.save_pretrained("lora_model")

In [26]:
if False: model.save_pretrained_merged("sos-llama", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("philipfourie/sos-llama", tokenizer, save_method = "merged_16bit")

In [27]:
if False: model.save_pretrained_gguf("sos-llama-gguf", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("philipfourie/sos-llama-gguf", tokenizer, quantization_method = "q4_k_m")